### Check how many layers are frozen in :

https://github.com/ShehabMMohamed/TinyImageNet-KaggleCompetition

In [ ]:
from keras.applications import Xception
from keras.layers import Input, Dense

In [ ]:
img_height = 299
img_width = 299

input_tensor = Input(shape=(img_width, img_height, 3))

pre_trained_model = Xception(weights='imagenet', input_tensor=input_tensor, include_top=False, pooling='avg')
# Downloads 83.7Mb of model..., 

In [ ]:
pre_trained_model.layers[20].name
# 'block3_sepconv2' ( next are 'block3_sepconv2_bn' and 'conv2d_2' )

In [ ]:
pre_trained_model.summary()
#Total params: 20,861,480
#Trainable params: 20,806,952
#Non-trainable params: 54,528
# Input ... block3_sepconv2 ... block14_sepconv2 ... global_average_pooling2d_1

In [ ]:
### That's a lot of fine tuning...

### Position of block3_sepconv2 in xception : 

https://github.com/keras-team/keras-applications/blob/master/keras_applications/xception.py#L188

### Should check the layout of the PyTorch model too...

https://github.com/Cadene/pretrained-models.pytorch/blob/master/pretrainedmodels/models/xception.py#L137

Seems to be only 12 blocks, which is a little odd...

### See UMAP results on plain MNIST raw image files

In [ ]:
! pip install umap-learn

In [ ]:
import numpy as np
from sklearn.datasets import load_iris, load_digits
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
%matplotlib inline

In [ ]:
sns.set(style='white', context='notebook', rc={'figure.figsize':(14,10)})

In [ ]:
digits = load_digits()
#print(digits.DESCR) # Documentation
#digits.images.shape # (1797, 8, 8)
digits.data.shape   # (1797, 64)

In [ ]:
import umap

In [ ]:
reducer = umap.UMAP(random_state=42)
reducer.fit(digits.data)  # <3

In [ ]:
embedding = reducer.transform(digits.data)
# Verify that the result of calling transform is
# idenitical to accessing the embedding_ attribute
assert(np.all(embedding == reducer.embedding_))
embedding.shape

In [ ]:
plt.scatter(embedding[:, 0], embedding[:, 1], c=digits.target, cmap='Spectral', s=5)
plt.gca().set_aspect('equal', 'datalim')
plt.colorbar(boundaries=np.arange(11)-0.5).set_ticks(np.arange(10))
plt.title('UMAP projection of the Digits dataset', fontsize=24);